In [1]:
!pip install torch torchvision

   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/203.1 MB 1.4 MB/s eta 0:02:21
   ---------------------------------------- 0.2/203.1 MB 1.8 MB/s eta 0:01:54
   ---------------------------------------- 0.3/203.1 MB 2.2 MB/s eta 0:01:34
   ---------------------------------------- 0.5/203.1 MB 2.5 MB/s eta 0:01:21
   ---------------------------------------- 0.9/203.1 MB 3.4 MB/s eta 0:00:59
   ---------------------------------------- 1.0/203.1 MB 3.8 MB/s eta 0:00:54
   ---------------------------------------- 1.3/203.1 MB 3.9 MB/s eta 0:00:53
   ---------------------------------------- 1.7/203.1 MB 4.6 MB/s eta 0:00:45
   ---------------------------------------- 2.0/203.1 MB 4.7 MB/s eta 0:00:44
   ---------------------------------------- 2.4/203.1 MB 5.3 MB/s eta 0:00:38
    --------------------------------------- 2.6/203.1 MB 5.4 MB/s eta 0:00:38



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\rishi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   -------------------------------- ------- 163.8/203.1 MB 5.9 MB/s eta 0:00:07
   -------------------------------- ------- 163.8/203.1 MB 5.9 MB/s eta 0:00:07
   -------------------------------- ------- 163.9/203.1 MB 5.5 MB/s eta 0:00:08
   -------------------------------- ------- 164.0/203.1 MB 5.4 MB/s eta 0:00:08
   -------------------------------- ------- 164.1/203.1 MB 5.4 MB/s eta 0:00:08
   -------------------------------- ------- 164.1/203.1 MB 5.4 MB/s eta 0:00:08
   -------------------------------- ------- 164.2/203.1 MB 5.0 MB/s eta 0:00:08
   -------------------------------- ------- 164.5/203.1 MB 5.0 MB/s eta 0:00:08
   -------------------------------- ------- 164.8/203.1 MB 5.0 MB/s eta 0:00:08
   -------------------------------- ------- 165.0/203.1 MB 4.9 MB/s eta 0:00:08
   -------------------------------- ------- 165.3/203.1 MB 4.9 MB/s eta 0:00:08
   -------------------------------- ------- 165.7/203.1 MB 5.0 MB/s eta 0:00:08
   -------------------------------- ---

In [3]:
# Example usage
input_dim = 100  # Latent vector dimension
output_channels = 1  # Number of channels in the output
base_dim = 64  # Base dimensionality of the model

# Initialize the generator
generator = ECGGenerator3D(input_dim, output_channels, base_dim)

# Create a random latent vector z (normally distributed)
z = tf.random.normal([1, input_dim])

# Generate the 3D ECG-like data
generated_data = generator(z)

print("Generated 3D Data Shape:", generated_data.shape)


Generated 3D Data Shape: (1, 32, 32, 32, 1)


In [2]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image

class ECGGenerator3D(tf.keras.Model):
    def __init__(self, input_dim, output_channels, base_dim):
        super(ECGGenerator3D, self).__init__()
        self.dense_layer = tf.keras.layers.Dense(base_dim * 4 * 4 * 4, activation='relu')
        self.reshape_layer = tf.keras.layers.Reshape((4, 4, 4, base_dim))  # Reshape to 3D volume
        
        self.deconv1 = tf.keras.layers.Conv3DTranspose(base_dim // 2, kernel_size=4, strides=2, padding='same', activation='relu')
        self.bn1 = tf.keras.layers.BatchNormalization()  # Batch Normalization layer for stable training
        
        self.deconv2 = tf.keras.layers.Conv3DTranspose(base_dim // 4, kernel_size=4, strides=2, padding='same', activation='relu')
        self.bn2 = tf.keras.layers.BatchNormalization()  # Batch Normalization
        
        self.deconv3 = tf.keras.layers.Conv3DTranspose(output_channels, kernel_size=4, strides=2, padding='same', activation='tanh')  # Output layer

    def call(self, z):
        x = self.dense_layer(z)  # Dense layer that outputs a flattened 3D volume
        x = self.reshape_layer(x)  # Reshape to match the initial volume size
        
        x = self.deconv1(x)  # Upsample to larger dimensions using deconvolution
        x = self.bn1(x)  # Batch Normalization
        
        x = self.deconv2(x)  # Further upsampling using deconvolution
        x = self.bn2(x)  # Batch Normalization
        
        return self.deconv3(x)  # Final output with 3D shape (ECG-like data)

def load_and_preprocess_data(local_folder_path):
    """
    Load and preprocess ECG images from the specified folder path.
    This function loads images, resizes them, and normalizes them.
    """
    ecg_images = []
    
    # List all files in the specified folder
    print(f"Looking for images in: {local_folder_path}")
    
    for file_name in os.listdir(local_folder_path):
        file_path = os.path.join(local_folder_path, file_name)
        
        # Print file names for debugging
        print(f"Found file: {file_name}")
        
        if file_name.endswith('.png'):  # Assuming ECG images are in PNG format
            try:
                image = Image.open(file_path).convert('L')  # Convert to grayscale
                image = image.resize((128, 128))  # Resize for uniformity
                image_array = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
                ecg_images.append(image_array)
            except Exception as e:
                print(f"Error loading image {file_name}: {e}")
    
    if len(ecg_images) == 0:
        raise FileNotFoundError("No valid ECG image files found in the specified folder.")
    
    # Stack the images to form the dataset
    ecg_images = np.stack(ecg_images)
    return ecg_images


def main():
    # Specify the correct local folder path for your ECG images
    local_folder_path = r"C:\Users\MTechS_h\Desktop\Mini Project\ecg_data\ECG_Image_data"
    
    # Load and preprocess data
    try:
        ecg_data = load_and_preprocess_data(local_folder_path)
    except FileNotFoundError as e:
        print(e)
        return

    print(f"Loaded ECG Data with shape: {ecg_data.shape}")

    # Define model parameters
    input_dim = 100  # Latent vector size
    output_channels = 1  # Single channel for ECG data
    base_dim = 64  # Base dimension for layers

    # Initialize the generator model
    generator = ECGGenerator3D(input_dim, output_channels, base_dim)

    # Create a random latent vector z
    z = tf.random.normal([1, input_dim])

    # Generate synthetic ECG-like data
    generated_data = generator(z)
    print("Generated 3D Data Shape:", generated_data.shape)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'tensorflow.python'